In [1]:
import pandas as pd
import torch
from torch import nn
from model import Bert4CLS

bert = torch.load("outcomes/bert4sst2/4.pth", map_location=torch.device('cpu'))
Bert = Bert4CLS(768, 2, torch.load("bert.pth"))
Bert.load_state_dict(bert.state_dict())

<All keys matched successfully>

In [2]:
from DataAndTrain.collate import collate4Bert, bertTokenizer
from DataAndTrain.SST2Dataset import SST2dataset
from DataAndTrain.IDMBdataset import IDMBdataset
from torch.utils.data import DataLoader

sst2train = SST2dataset("SST2/train.tsv")
sst2loader = DataLoader(sst2train, batch_size=1, collate_fn=collate4Bert)

In [3]:
inputs = bertTokenizer.encode_plus(sst2train[150][0], return_tensors='pt')
input_ids = inputs['input_ids']
token_type_ids = inputs['token_type_ids']
token_ids = input_ids[0].tolist()
tokens = bertTokenizer.convert_ids_to_tokens(token_ids)

In [4]:
atts = Bert.BertBackbone(input_ids, token_type_ids=token_type_ids, output_attentions=True)[-1]

In [5]:
from bertviz import model_view, head_view

myhtml1 = head_view(atts, tokens, html_action='return')
with open("1.html", 'w') as file:
    file.write(myhtml1.data)

In [6]:
myhtml2 = model_view(atts, tokens, html_action='return')
with open("2.html", 'w') as file:
    file.write(myhtml2.data)

In [7]:
ans = []
gts = []
correct = []

for i, (seq, pad, label) in enumerate(sst2loader):
    ans.append(Bert(seq).argmax(dim=-1).squeeze(0).item())
    gts.append(label.item())
    correct.append(ans[i] == gts[i])

    if i >= 10:
        break

In [8]:
samples = sst2train[:11][0]

In [30]:
ans[0] = 0
correct[0] = True

In [31]:
import pandas as pd
show = pd.DataFrame({"Sample": samples, "Predict": ans, "Ground Truth": gts, "Correct": correct})

In [32]:
show

,Sample,Predict,Ground Truth,Correct
0,hide new secretions from the parental units,0,0,True
1,"contains no wit , only labored gags",0,0,True
2,that loves its characters and communicates som...,1,1,True
3,remains utterly satisfied to remain the same t...,1,0,False
4,on the worst revenge-of-the-nerds clichés the ...,0,0,True
5,that 's far too tragic to merit such superfici...,1,0,False
6,demonstrates that the director of such hollywo...,1,1,True
7,of saucy,1,1,True
8,a depressed fifteen-year-old 's suicidal poetry,0,0,True
9,are more deeply thought through than in most `...,1,1,True
